In [1]:
!export HF_HOME=/vol/bitbucket/rm1623/.cache/

In [2]:
import os 
os.environ["HF_HOME"] = "/vol/bitbucket/rm1623/.cache/"


In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

import wandb
import torch 
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

/homes/rm1623/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]


In [3]:
model_name = 'facebook/opt-350m'
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear(in_features=1024, out_features=512, bias=False)
      (project_in): Linear(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=409

In [54]:
# model.config

In [5]:
dataset = load_dataset("tatsu-lab/alpaca")
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 52002
    })
})

In [6]:
tokenizer.pad_token = tokenizer.eos_token
# model.resize_token_embeddings(len(tokenizer))

In [63]:
import math

vocab_size = model.config.vocab_size
bit_length = math.ceil(math.log2(vocab_size))
vocab_size, bit_length, 2**bit_length

(50272, 16, 65536)

In [64]:
class BitStringOutputLayer(nn.Module):
    def __init__(self, hidden_dim, bit_length):
        super(BitStringOutputLayer, self).__init__()
        self.linear = nn.Linear(hidden_dim, bit_length)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        return self.sigmoid(self.linear(x))
    
# model.lm_head = BitStringOutputLayer(model.config.hidden_size, bit_length)
model.lm_head = BitStringOutputLayer(model.config.word_embed_proj_dim, bit_length)
model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear(in_features=1024, out_features=512, bias=False)
      (project_in): Linear(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=409

## OPTForCausalLM

In [76]:
from dataclasses import dataclass
from typing import Optional, Tuple, Union, List

@dataclass
class CausalLMOutputWithPast():
    """
    Base class for causal language model (or autoregressive) outputs.

    Args:
        loss (`torch.FloatTensor` of shape `(1,)`, *optional*, returned when `labels` is provided):
            Language modeling loss (for next-token prediction).
        logits (`torch.FloatTensor` of shape `(batch_size, sequence_length, config.vocab_size)`):
            Prediction scores of the language modeling head (scores for each vocabulary token before SoftMax).
        past_key_values (`tuple(tuple(torch.FloatTensor))`, *optional*, returned when `use_cache=True` is passed or when `config.use_cache=True`):
            Tuple of `tuple(torch.FloatTensor)` of length `config.n_layers`, with each tuple having 2 tensors of shape
            `(batch_size, num_heads, sequence_length, embed_size_per_head)`)

            Contains pre-computed hidden-states (key and values in the self-attention blocks) that can be used (see
            `past_key_values` input) to speed up sequential decoding.
        hidden_states (`tuple(torch.FloatTensor)`, *optional*, returned when `output_hidden_states=True` is passed or when `config.output_hidden_states=True`):
            Tuple of `torch.FloatTensor` (one for the output of the embeddings, if the model has an embedding layer, +
            one for the output of each layer) of shape `(batch_size, sequence_length, hidden_size)`.

            Hidden-states of the model at the output of each layer plus the optional initial embedding outputs.
        attentions (`tuple(torch.FloatTensor)`, *optional*, returned when `output_attentions=True` is passed or when `config.output_attentions=True`):
            Tuple of `torch.FloatTensor` (one for each layer) of shape `(batch_size, num_heads, sequence_length,
            sequence_length)`.

            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention
            heads.
    """

    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    past_key_values: Optional[Tuple[Tuple[torch.FloatTensor]]] = None
    hidden_states: Optional[Tuple[torch.FloatTensor, ...]] = None
    attentions: Optional[Tuple[torch.FloatTensor, ...]] = None


In [77]:
from transformers import OPTPreTrainedModel, OPTModel

In [79]:
config = {
  "_name_or_path": "opt-350m",
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "architectures": [
    "OPTForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "do_layer_norm_before": False,
  "dropout": 0.1,
  "eos_token_id": 2,
  "ffn_dim": 4096,
  "hidden_size": 1024,
  "init_std": 0.02,
  "layerdrop": 0.0,
  "max_position_embeddings": 2048,
  "model_type": "opt",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "prefix": "</s>",
  "torch_dtype": "float16",
  "transformers_version": "4.20.0.dev0",
  "use_cache": True,
  "vocab_size": 50272,
  "word_embed_proj_dim": 512
}

In [81]:
optmodel = OPTModel.from_pretrained("facebook/opt-350m")

In [83]:
sample_output = optmodel(**tokenizer("Hello, my dog is cute", return_tensors="pt"))


In [87]:
len(tokenizer("Hello, my dog is cute", return_tensors="pt").input_ids[0])

7

In [95]:
labels = tokenizer("Hello, my dog is cute", return_tensors="pt").input_ids
labels

tensor([[    2, 31414,     6,   127,  2335,    16, 11962]])

In [84]:
sample_output[0].shape

torch.Size([1, 7, 512])

In [90]:
lm_head = nn.Linear(512, 16, bias=False)

logits = lm_head(sample_output[0])

In [94]:
shift_logits = logits[..., :-1, :].contiguous()
shift_labels = labels[..., 1:].contiguous()
shift_logits.shape, shift_labels.shape

(torch.Size([1, 6, 16]), torch.Size([1, 6]))

In [97]:
shift_logits[0][0]

tensor([-2.2454,  5.2966, -1.3153, -0.4532, -0.8342,  0.5373, -0.4919,  1.9976,
        -3.5326,  2.9077,  1.4376, -1.5307, -0.4819,  2.5617, -3.1072, -0.5668],
       grad_fn=<SelectBackward0>)

In [96]:
shift_labels

tensor([[31414,     6,   127,  2335,    16, 11962]])

In [ ]:

class OPTForCausalLM(OPTPreTrainedModel):
    _tied_weights_keys = ["lm_head.weight"]

    def __init__(self, config):
        super().__init__(config)
        self.model = OPTModel(config)

        # the lm_head weight is automatically tied to the embed tokens weight
        self.bit_size = torch.log2(torch.tensor(config.vocab_size)).ceil().int().item()
        self.lm_head = nn.Linear(config.word_embed_proj_dim, self.bit_size, bias=False)

        # Initialize weights and apply final processing
        self.post_init()

    def get_input_embeddings(self):
        return self.model.decoder.embed_tokens

    def set_input_embeddings(self, value):
        self.model.decoder.embed_tokens = value

    def get_output_embeddings(self):
        return self.lm_head

    def set_output_embeddings(self, new_embeddings):
        self.lm_head = new_embeddings

    def set_decoder(self, decoder):
        self.model.decoder = decoder

    def get_decoder(self):
        return self.model.decoder

    def forward(
        self,
        input_ids: torch.LongTensor = None,
        attention_mask: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, CausalLMOutputWithPast]:
        r"""
        """

        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        # decoder outputs consists of (dec_features, layer_state, dec_hidden, dec_attn)
        outputs = self.model.decoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            head_mask=head_mask,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        logits = self.lm_head(outputs[0]).contiguous()
        
        

        loss = None
        if labels is not None:
            # move labels to correct device to enable model parallelism
            labels = labels.to(logits.device)
            # Shift so that tokens < n predict n
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            # Flatten the tokens
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(shift_logits.view(-1, self.config.vocab_size), shift_labels.view(-1))

        if not return_dict:
            output = (logits,) + outputs[1:]
            return (loss,) + output if loss is not None else output

        return CausalLMOutputWithPast(
            loss=loss,
            logits=logits,
            past_key_values=outputs.past_key_values,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

    def prepare_inputs_for_generation(
        self, input_ids, past_key_values=None, attention_mask=None, inputs_embeds=None, **kwargs
    ):
        if past_key_values is not None:
            past_length = past_key_values[0][0].shape[2]

            # Some generation methods already pass only the last input ID
            if input_ids.shape[1] > past_length:
                remove_prefix_length = past_length
            else:
                # Default to old behavior: keep only final ID
                remove_prefix_length = input_ids.shape[1] - 1

            input_ids = input_ids[:, remove_prefix_length:]

        # if `inputs_embeds` are passed, we only want to use them in the 1st generation step
        if inputs_embeds is not None and past_key_values is None:
            model_inputs = {"inputs_embeds": inputs_embeds}
        else:
            model_inputs = {"input_ids": input_ids}

        model_inputs.update(
            {
                "past_key_values": past_key_values,
                "use_cache": kwargs.get("use_cache"),
                "attention_mask": attention_mask,
            }
        )
        return model_inputs

    @staticmethod
    def _reorder_cache(past_key_values, beam_idx):
        reordered_past = ()
        for layer_past in past_key_values:
            reordered_past += (
                tuple(past_state.index_select(0, beam_idx.to(past_state.device)) for past_state in layer_past),
            )
        return reordered_past


In [ ]:
import torch

def hamming_loss(predicted, target):
    """Compute the Hamming loss between predicted and target bit strings"""
    return (predicted != target).float().mean()



In [7]:
from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling
import numpy as np

In [14]:
sample_dataset = dataset['train'].select(range(5))

In [8]:
texts = dataset["train"]["text"]

In [15]:
def tokenize_function(examples):
        return tokenizer(examples["text"])

In [24]:
sample_dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 5
})

In [30]:
texts = dataset["train"]["text"]
block_size = 10
sample_tokenized_datasets = sample_dataset.map(
            tokenize_function,
            batched=True,
            num_proc=1,
            remove_columns=['instruction', 'input', 'output', 'text'],
            load_from_cache_file=False,
            desc="Running tokenizer on dataset",
        )


Running tokenizer on dataset: 100%|██████████| 5/5 [00:00<00:00, 307.38 examples/s]


In [31]:
sample_tokenized_datasets

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 5
})

In [32]:
from itertools import chain 

def group_texts(examples):
        # Concatenate all texts.
        concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
        total_length = len(concatenated_examples[list(examples.keys())[0]])
        # We drop the small remainder, and if the total_length < block_size  we exclude this batch and return an empty dict.
        # We could add padding if the model supported it instead of this drop, you can customize this part to your needs.
        total_length = (total_length // block_size) * block_size
        # Split by chunks of max_len.
        result = {
            k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
            for k, t in concatenated_examples.items()
        }
        result["labels"] = result["input_ids"].copy()
        print(result)
        return result

In [33]:
sample_lm_datasets = sample_tokenized_datasets.map(
            group_texts,
            batched=True,
            num_proc=1,
            load_from_cache_file=False,
            desc=f"Grouping texts in chunks of {block_size}",
        )

Grouping texts in chunks of 10: 100%|██████████| 5/5 [00:00<00:00, 306.96 examples/s]

{'input_ids': [[2, 45943, 16, 41, 15741, 14, 7448, 10, 3685, 4], [21062, 10, 1263, 14, 16574, 25830, 5, 2069, 4, 50118], [50118, 48134, 41241, 35, 50118, 31033, 130, 4965, 13, 4959], [2245, 4, 50118, 50118, 48134, 19121, 35, 50118, 134, 4], [43800, 10, 9320, 5626, 8, 146, 686, 7, 680, 2710], [9, 12849, 8, 8942, 4, 1437, 50118, 176, 4, 30450], [4595, 7, 489, 110, 809, 2171, 8, 670, 4, 1437], [50118, 246, 4, 2315, 615, 3581, 8, 3014, 10, 4292], [3581, 3078, 4, 2, 45943, 16, 41, 15741, 14, 7448], [10, 3685, 4, 21062, 10, 1263, 14, 16574, 25830, 5], [2069, 4, 50118, 50118, 48134, 41241, 35, 50118, 2264, 32], [5, 130, 2270, 8089, 116, 50118, 50118, 48134, 19121, 35], [50118, 133, 130, 2270, 8089, 32, 1275, 6, 2440, 6], [8, 5718, 4, 2, 45943, 16, 41, 15741, 14, 7448], [10, 3685, 4, 21062, 10, 1263, 14, 16574, 25830, 5], [2069, 4, 50118, 50118, 48134, 41241, 35, 50118, 47066, 21700], [5, 3184, 9, 41, 37113, 4, 50118, 50118, 48134, 19121], [35, 50118, 4688, 37113, 16, 156, 62, 9, 10, 38531], [

In [34]:
sample_lm_datasets

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 55
})

In [42]:
from transformers import default_data_collator, get_scheduler

In [36]:
train_dataloader = DataLoader(
        sample_lm_datasets, shuffle=True, collate_fn=default_data_collator, batch_size=1
    )

In [37]:
no_decay = ["bias", "layer_norm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.001,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=1e-4)


In [44]:
num_train_epochs = 2
gradient_accumulation_steps = 2
num_warmup_steps = 5
num_processes = 1

In [65]:
import math
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
max_train_steps = num_train_epochs * num_update_steps_per_epoch
overrode_max_train_steps = True


In [66]:
lr_scheduler = get_scheduler(
        name="linear",
        optimizer=optimizer,
        num_warmup_steps=num_warmup_steps * num_processes,
        num_training_steps=max_train_steps
)

In [67]:
from accelerate import Accelerator, DistributedType
from accelerate.logging import get_logger
from accelerate.utils import set_seed

In [68]:
accelerator = Accelerator(gradient_accumulation_steps=gradient_accumulation_steps)


In [69]:
model, optimizer, train_dataloader, eval_dataloader, lr_scheduler = accelerator.prepare(
        model, optimizer, train_dataloader, None, lr_scheduler
    )

In [70]:
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
max_train_steps = num_train_epochs * num_update_steps_per_epoch
num_train_epochs = math.ceil(max_train_steps / num_update_steps_per_epoch)

In [71]:
checkpointing_steps = 100
per_device_train_batch_size = 1

total_batch_size = per_device_train_batch_size * num_processes * gradient_accumulation_steps

In [72]:
from tqdm import tqdm 
with_tracking = True

resume_from_checkpoint = False
starting_epoch=0
completed_steps = 0

resume_step = None

progress_bar = tqdm(range(max_train_steps), disable=not accelerator.is_local_main_process)
per_device_eval_batch_size = 1

57it [03:53,  4.09s/it]


In [73]:
for epoch in range(starting_epoch, num_train_epochs):
        model.train()
        if with_tracking:
            total_loss = 0
        if resume_from_checkpoint and epoch == starting_epoch and resume_step is not None:
            # We skip the first `n` batches in the dataloader when resuming from a checkpoint
            active_dataloader = accelerator.skip_first_batches(train_dataloader, resume_step)
        else:
            active_dataloader = train_dataloader
        for step, batch in enumerate(active_dataloader):
            with accelerator.accumulate(model):
                outputs = model(**batch)
                loss = outputs.loss
                # We keep track of the loss at each epoch
                if with_tracking:
                    total_loss += loss.detach().float()
                accelerator.backward(loss)
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

            # Checks if the accelerator has performed an optimization step behind the scenes
            if accelerator.sync_gradients:
                progress_bar.update(1)
                completed_steps += 1

            if isinstance(checkpointing_steps, int):
                if completed_steps % checkpointing_steps == 0:
                    output_dir = f"step_{completed_steps}"
                    accelerator.save_state(output_dir)
            if completed_steps >= max_train_steps:
                break

        model.eval()
        losses = []
        if eval_dataloader is not None:
            for step, batch in enumerate(eval_dataloader):
                with torch.no_grad():
                    outputs = model(**batch)

                loss = outputs.loss
                losses.append(accelerator.gather_for_metrics(loss.repeat(per_device_eval_batch_size)))

            losses = torch.cat(losses)
            try:
                eval_loss = torch.mean(losses)
                perplexity = math.exp(eval_loss)
            except OverflowError:
                perplexity = float("inf")

            print(f"epoch {epoch}: perplexity: {perplexity} eval_loss: {eval_loss}")

            if with_tracking:
                accelerator.log(
                    {
                        "perplexity": perplexity,
                        "eval_loss": eval_loss,
                        "train_loss": total_loss.item() / len(train_dataloader),
                        "epoch": epoch,
                        "step": completed_steps,
                    },
                    step=completed_steps,
                )


if with_tracking:
    accelerator.end_training()

# if args.output_dir is not None:
#     accelerator.wait_for_everyone()
#     unwrapped_model = accelerator.unwrap_model(model)
#     unwrapped_model.save_pretrained(
#         args.output_dir, is_main_process=accelerator.is_main_process, save_function=accelerator.save
#     )
#     if accelerator.is_main_process:
#         tokenizer.save_pretrained(args.output_dir)
#         if args.push_to_hub:
#             api.upload_folder(
#                 commit_message="End of training",
#                 folder_path=args.output_dir,
#                 repo_id=repo_id,
#                 repo_type="model",
#                 token=args.hub_token,
#             )
#         with open(os.path.join(args.output_dir, "all_results.json"), "w") as f:
#             json.dump({"perplexity": perplexity}, f)



RuntimeError: shape '[-1, 50272]' is invalid for input of size 144

## Explore

In [64]:
example_outputs = model(input_ids[:2])

In [65]:
example_outputs.logits[0].shape

torch.Size([1511, 16])

In [66]:
example_outputs.logits[0]

tensor([[0.6652, 0.3778, 0.0269,  ..., 0.0633, 0.9739, 0.5414],
        [0.3898, 0.5647, 0.1780,  ..., 0.8867, 0.0281, 0.7063],
        [0.4054, 0.6966, 0.8962,  ..., 0.9953, 0.0642, 0.9606],
        ...,
        [0.7411, 0.1787, 0.0168,  ..., 0.0109, 0.9965, 0.8426],
        [0.7360, 0.1742, 0.0163,  ..., 0.0109, 0.9966, 0.8393],
        [0.7358, 0.1741, 0.0164,  ..., 0.0107, 0.9966, 0.8414]],
       grad_fn=<SelectBackward0>)

In [49]:
input_ids[0]

torch.Size([1511])

In [68]:
def to_binary(x):
    return x > 0.5  # 0.5 is the threshold

# to_binary(example_outputs.logits[0])

In [ ]:
def prepare_dataset_for_mlm(texts):
    inputs = tokenizer(texts, padding=True, return_tensors="pt", truncation=True, max_length=128)
    input_ids = inputs["input_ids"]
    attention_masks = inputs["attention_mask"]
    
    rng = np.random.default_rng()  # Random number generator
    mask_arr = rng.random(input_ids.shape) < 0.15  # Mask 15% of all tokens
    mask_arr = mask_arr & (input_ids != tokenizer.pad_token_id)  # Do not mask padding tokens

    # Replace 80% of masked positions with [MASK] token, 10% with random token, 10% unchanged
    mask_positions = np.where(mask_arr)
    masked_input_ids = input_ids.clone()
    for i in range(mask_positions[0].size):
        rand = rng.random()
        if rand < 0.8:  # 80% replaced by [MASK]
            masked_input_ids[mask_positions[0][i], mask_positions[1][i]] = tokenizer.mask_token_id
        elif rand < 0.9:  # 10% replaced by random token
            masked_input_ids[mask_positions[0][i], mask_positions[1][i]] = rng.integers(1, tokenizer.vocab_size)
    labels = -100 * torch.ones(input_ids.shape, dtype=torch.long)
    labels[mask_arr] = input_ids[mask_arr]

In [31]:
# Freeze the existing parameters for the base model 
for param in model.base_model.parameters():
    param.requires_grad = False
# 

In [33]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
model.train()

In [ ]:
num_epochs = 2

In [13]:
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(input_ids)
    predictions = model.lm_head(outputs.last_hidden_state)
    loss = loss_fn(predictions, labels)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch} Loss: {loss.item()}")

In [41]:
model.config.hidden_size, bit_length

(4096, 15)

In [14]:
input_ids = torch.randint(0, vocab_size, (1, 10))  # Example input
outputs = model(input_ids)

In [23]:
optimizer = optim.Adam(model.parameters(), lr=0.001)